# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input, Concatenate
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle
import glob
from keras.utils import to_categorical
from keras import regularizers

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/4. normalized-50/'
data_directory_20 = 'dataset/3. Normalized/'
data_directory_30 = 'dataset/5. normalized-30/'
data_directory_10 = 'dataset/6. normalized-10/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))

x_train_20 = pickle.load(open(data_directory_20 + "x_train.p", "rb"))
y_train_20 = pickle.load(open(data_directory_20 + "y_train.p", "rb"))
x_test_20 = pickle.load(open(data_directory_20 + "x_test.p", "rb"))
y_test_20 = pickle.load(open(data_directory_20 + "y_test.p", "rb"))

x_train_30 = pickle.load(open(data_directory_30 + "x_train.p", "rb"))
y_train_30 = pickle.load(open(data_directory_30 + "y_train.p", "rb"))
x_test_30 = pickle.load(open(data_directory_30 + "x_test.p", "rb"))
y_test_30 = pickle.load(open(data_directory_30 + "y_test.p", "rb"))

x_train_10 = pickle.load(open(data_directory_10 + "x_train.p", "rb"))
y_train_10 = pickle.load(open(data_directory_10 + "y_train.p", "rb"))
x_test_10 = pickle.load(open(data_directory_10 + "x_test.p", "rb"))
y_test_10 = pickle.load(open(data_directory_10 + "y_test.p", "rb"))

print('x_train: ' + str(x_train_10.shape))
print('y_train: ' + str(y_train_10.shape))
print('x_test: ' + str(x_test_10.shape))
print('y_test: ' + str(y_test_10.shape))

x_train: (1087, 26)
y_train: (1087, 2)
x_test: (121, 26)
y_test: (121, 2)


# Load auto encoders

In [3]:
autoencoder_1_path = 'results/46-autoencoder-1-10-01/model/2018-03-11-18-14-44/weights.hdf5'
autoencoder_2_path = 'results/47-autoencoder-2-10-01/model/2018-03-11-20-18-49/weights.hdf5'
weights_1 = load_model(autoencoder_1_path)
weights_1.summary()
weights_2 = load_model(autoencoder_2_path)
weights_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                270       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 26)                286       
Total params: 556
Trainable params: 556
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_2 (Den

In [4]:
x_train_ae = weights_1.predict(x_train_10)
print(x_train_ae.shape)
x_train_10 = np.vstack([x_train_10, x_train_ae])
print(x_train_10.shape)
y_train_10 = np.vstack([y_train_10, y_train_10])
print(y_train_10.shape)

(1087, 26)
(2174, 26)
(2174, 2)


# Set up constants for weight transfer

In [5]:
autoencoder_1_layer_units = 10
autoencoder_1_layer_name = 'dense_1'
autoencoder_1_layer_dropout = 0

autoencoder_2_layer_units = 8
autoencoder_2_layer_name = 'dense_2'
autoencoder_2_layer_dropout = 0

# Tubable parameters

In [6]:
epochs = 100000
batch_size = 20
learning_rate = 0.00005
decay = 0.00001
loss = keras.losses.categorical_crossentropy

# Model for classification

In [7]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/48-resnet-2-10-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"

def create_model(layers, dropout, test_size, regularization):
    global date
    global autoencoder_path
    x = Input(shape=(x_train.shape[1],))
    y = Dense(units = autoencoder_1_layer_units, activation='relu')(x)
    y = Dropout(autoencoder_1_layer_dropout)(y)
    y = Dense(units = autoencoder_2_layer_units, activation='relu')(y)
    y = Dropout(autoencoder_1_layer_dropout)(y)
    y2 = Concatenate()([y, x])
    y = Activation('relu')(y2)
    
    y = Dense(units = layers['layer_1'], activation='relu', kernel_regularizer=regularizers.l2(regularization))(y)
    y = Dropout(dropout)(y)
    
    if (layers['layer_2'] > 0):
        y = Dense(units = layers['layer_2'], activation='relu', kernel_regularizer=regularizers.l2(regularization))(y)
        y = Dropout(dropout)(y)
    
        if (layers['layer_3'] > 0):
            y = Dense(units = layers['layer_3'], activation='relu', kernel_regularizer=regularizers.l2(regularization))(y)
            y = Dropout(dropout)(y)
            
    y = Concatenate()([y, y2, x])
    y = Activation('relu')(y)
    
    y = Dense(units = layers['layer_4'], activation='relu', kernel_regularizer=regularizers.l2(regularization))(y)
    y = Dropout(dropout)(y)
    
    if (layers['layer_5'] > 0):
        y = Dense(units = layers['layer_5'], activation='relu', kernel_regularizer=regularizers.l2(regularization))(y)
#         y = Dropout(dropout)(y)
    
    y = Dense(units = y_train.shape[1], activation='softmax')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Layer 1': layers['layer_1'],
        'Layer 2': layers['layer_2'],
        'Layer 3': layers['layer_3'],
        'Layer 4': layers['layer_4'],
        'Layer 5': layers['layer_5'],
        'Dropout all': dropout,
        'Learning rate': learning_rate,
        'Decay': decay,
        'Batch size': batch_size,
        'Autoencoder 1': autoencoder_1_path,
        'Autoencoder 2': autoencoder_2_path,
        'Epochs': epochs,
        'Fine tune': True,
        'Loss': loss,
        'Test data size (percentage)': test_size,
        'Regularization': regularization,
        'Use data augmentation': True
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=True)
    SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=loss, metrics=['accuracy'])
    
    # Set layer weights to pre trained autoencoder weights
    model.layers[1].set_weights(weights_1.get_layer(autoencoder_1_layer_name).get_weights())
    model.layers[3].set_weights(weights_2.get_layer(autoencoder_2_layer_name).get_weights())
    
    return model

# Prepare callbacks

In [ ]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=False)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, verbose=0, **kwargs)

# Train

In [ ]:
test_sizes = [10]
for test_size in test_sizes:
    model = None
    model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

    layers = [
        {'layer_1': 128, 'layer_2': 0, 'layer_3': 0, 'layer_4': 256, 'layer_5': 0}
    ]
    
    param_grid = dict(
        layers=layers,
        dropout=[0.5],
        test_size=[test_size],
        regularization=[0.004]
    )
    
    if (test_size == 50):
        x_train_final = x_train
        x_test_final = x_test
        y_train_final = y_train
        y_test_final = y_test
    elif (test_size == 30):
        x_train_final = x_train_30
        x_test_final = x_test_30
        y_train_final = y_train_30
        y_test_final = y_test_30
    elif (test_size == 10):
        x_train_final = x_train_10
        x_test_final = x_test_10
        y_train_final = y_train_10
        y_test_final = y_test_10
    else:
        x_train_final = x_train_20
        x_test_final = x_test_20
        y_train_final = y_train_20
        y_test_final = y_test_20

    grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test_final, y_test_final)))
    grid_result = grid.fit(x_train_final, y_train_final)

Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV] regularization=0.004, test_size=10, dropout=0.5, layers={'layer_1': 128, 'layer_3': 0, 'layer_2': 0, 'layer_5': 0, 'layer_4': 256} 
